In [ ]:
!nvidia-smi

Tue May 31 01:08:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    42W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q transformers

In [ ]:
import os, sys
from pathlib import Path
from tqdm.notebook import tqdm


import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import RobertaForMaskedLM, RobertaTokenizerFast, RobertaTokenizer
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

# Load clean dataset

In [ ]:
# df = pd.read_pickle('/content/drive/MyDrive/NLP/AI4Code/dataset/exp_5/clean_dataset.pkl')
# df

# Prepare dataset for Mask Language Model

In [ ]:
# df_test = df.loc[:976].copy()
# df_test.tail()

In [ ]:
# with open('/content/drive/MyDrive/NLP/AI4Code/dataset/exp_5/corpus_mlm_codebert.txt', 'w') as f:
#     for id, item in tqdm(df.groupby('id')):
#         df_markdown = item[item['cell_type'] == 'markdown']
#         df_code = item[item['cell_type'] == 'code']

#         matched_cid = []
#         for cid, source, rank in df_markdown[['cell_id', 'source', 'rank']].values:
#             cell_source = df_code[df_code['rank'] == (rank+1)]
            
#             if len(cell_source):
#                 sentence = source + "</s>" + cell_source.source.values[0]
#                 matched_cid.append(cid)
#                 matched_cid.append(cell_source['cell_id'].values[0])
#                 f.write(sentence+'\n')
        
#         for cid, source, ctype in item[['cell_id', 'source', 'cell_type']].values:
#             if cid not in matched_cid:
#                 if ctype == 'markdown':
#                     sentence = source + "</s>" +  ''
#                 elif ctype == 'code':
#                     sentence = '' + "</s>" +  source
#                 else:
#                     raise Exception
#                 f.write(sentence+'\n')

# Train model



In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained("microsoft/codebert-base", do_lower_case=True)
# tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base", do_lower_case=True)
model = RobertaForMaskedLM.from_pretrained("microsoft/codebert-base")

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
text = "all image have same shape</s>"
code_tokens=tokenizer.tokenize(text, add_special_tokens=True)
code_tokens

['<s>', 'all', 'Ġimage', 'Ġhave', 'Ġsame', 'Ġshape', '</s>', '</s>']

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path = '/content/drive/MyDrive/NLP/AI4Code/dataset/exp_5/corpus_mlm_codebert.txt',
    block_size = 128
)

print("No. of lines: ", len(dataset))

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


No. of lines:  4791671


In [ ]:
training_args = TrainingArguments(
    output_dir = '/content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_mlm',
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    logging_first_step=True,
    logging_steps=5000,
    save_steps=10000,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# trainer.train()
trainer.train(resume_from_checkpoint = True)

Loading model from /content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_mlm/checkpoint-110000).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4791671
  Num Epochs = 5
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 187175
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 2
  Continuing training from global step 110000
  Will skip the first 2 epochs then the first 35130 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, bu

  0%|          | 0/35130 [00:00<?, ?it/s]

Step,Training Loss
115000,0.823800
120000,0.807500
125000,0.797500
130000,0.793000
135000,0.785600
140000,0.777800


Saving model checkpoint to /content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_mlm/checkpoint-120000
Configuration saved in /content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_mlm/checkpoint-120000/config.json
Model weights saved in /content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_mlm/checkpoint-120000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_mlm/checkpoint-130000
Configuration saved in /content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_mlm/checkpoint-130000/config.json
Model weights saved in /content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_mlm/checkpoint-130000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_mlm/checkpoint-140000
Configuration saved in /content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_mlm/checkpoint-140000/config.json
Model weights saved in /content/drive/MyDrive/NLP/AI

Step,Training Loss
115000,0.823800
120000,0.807500
125000,0.797500
130000,0.793000
135000,0.785600
140000,0.777800
145000,0.776000
150000,0.768200
155000,0.762700
160000,0.759100


Saving model checkpoint to /content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_mlm/checkpoint-150000
Configuration saved in /content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_mlm/checkpoint-150000/config.json
Model weights saved in /content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_mlm/checkpoint-150000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_mlm/checkpoint-160000
Configuration saved in /content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_mlm/checkpoint-160000/config.json
Model weights saved in /content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_mlm/checkpoint-160000/pytorch_model.bin


KeyboardInterrupt: ignored

In [ ]:
# trainer.save_model('/content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_mlm')